```mermaid
flowchart TD
    A[User uploads PDF manual] --> B[Chunk + Embed PDF content]
    B --> C[Upload embeddings to Vertex AI Vector Index]
    C --> D[Create public Index Endpoint]

    subgraph Vertex AI
        C
        D
    end

    E[User asks a question about the Acura MDX] --> F[Query Vertex AI Matching Engine]
    F --> G[Retrieve relevant chunks]

    G --> H[Gemini LLM generates response]
    H --> I[Show Answer to User]

    %% Crew AI Integration
    subgraph Crew AI Agents
        J[Weather Agent - Winter]
        K[Manual Review Agent]
        L[Recommendation Agent]
    end

    J --> M[Inject seasonal context]
    G --> K
    K --> L
    L --> H
```

In [ ]:
# !pip install google-cloud-storage
# 🧱 Cell 1: Setup environment and initialize Vertex AI

import os
from dotenv import load_dotenv, find_dotenv
from google.cloud import aiplatform
import vertexai

# Load environment variables
load_dotenv(find_dotenv(), override=True)

# Set ADC to correct service account
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GCP_KEY_PATH")

# Init Vertex AI under the correct project
project_id = os.getenv("GCP_PROJECT_ID")
vertexai.init(project=project_id, location=os.getenv("VERTEX_REGION"))
aiplatform.init(project=project_id, location=os.getenv("VERTEX_REGION"))

print(f"✅ Vertex AI initialized in project: {project_id}")

In [ ]:
# 🧱 Cell 2 (refactored): Recreate correct GCS bucket and upload Acura MDX PDF

import os
from dotenv import load_dotenv
from google.cloud import storage

# Load .env variables
load_dotenv()

GCP_PROJECT_ID = os.getenv("GCP_PROJECT_ID")
GCS_BUCKET_NAME = os.getenv("GCS_BUCKET_NAME")
GCS_BUCKET_REGION = os.getenv("GCS_BUCKET_REGION")
PDF_LOCAL_PATH = os.getenv("PDF_LOCAL_PATH")
GCS_DEST_PATH = os.getenv("GCS_DEST_PATH")


# ✅ Create GCS bucket (no deprecated warning)
def create_bucket(bucket_name, location):
    try:
        bucket = client.lookup_bucket(bucket_name)
        if bucket:
            print(f"✅ Bucket '{bucket_name}' already exists.")
        else:
            print(f"📦 Creating bucket: {bucket_name} in region: {location} ...")
            bucket = client.bucket(bucket_name)
            new_bucket = client.create_bucket(bucket, location=location)
            print(f"✅ Bucket '{bucket_name}' created in {location}.")
    except Exception as e:
        print(f"❌ Error creating bucket: {e}")

# Upload PDF to GCS
def upload_file_to_gcs(bucket_name, source_path, destination_blob):
    # print(f"📤 Uploading '{source_path}' to 'gs://{bucket_name}/{destination_blob}' ...")
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob)
    blob.upload_from_filename(source_path)
    print(f"✅ Upload complete.")

# Execute
create_bucket(GCS_BUCKET_NAME, GCS_BUCKET_REGION)
upload_file_to_gcs(GCS_BUCKET_NAME, PDF_LOCAL_PATH, GCS_DEST_PATH)

In [ ]:
# 🧱 Cell 3 (rewritten): Layout-aware chunking using PyMuPDF for better accuracy

# !pip install --upgrade pymupdf
import fitz  # This is PyMuPDF (not the wrong 'fitz' package)

def extract_and_chunk_pdf_layout(pdf_path, max_chars=2000, overlap_chars=200):
    print(f"📥 Loading PDF: {pdf_path}")
    doc = fitz.open(pdf_path)
    print(f"📄 PDF has {len(doc)} pages.")

    chunks = []
    current_chunk = ""
    metadata = []

    for page_num, page in enumerate(doc, start=1):
        text = page.get_text("text")
        if not text or len(text.strip()) < 100:
            continue  # skip blank or low-content pages

        lines = text.strip().splitlines()
        for line in lines:
            line = line.strip()
            if not line:
                continue
            if len(current_chunk) + len(line) + 1 > max_chars:
                chunks.append(current_chunk.strip())
                metadata.append({"page": page_num})
                current_chunk = current_chunk[-overlap_chars:] + "\n" + line
            else:
                current_chunk += "\n" + line

    if current_chunk:
        chunks.append(current_chunk.strip())
        metadata.append({"page": len(doc)})

    print(f"✅ Chunking complete: {len(chunks)} layout-aware chunks created.")
    return chunks, metadata

# Use the local path from .env
pdf_path = os.getenv("PDF_LOCAL_PATH")
text_chunks, chunk_metadata = extract_and_chunk_pdf_layout(pdf_path)

In [ ]:
# 🧱 Cell 4: Embed text chunks using sentence-transformers (local) with incremental logging

# !pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer
import uuid

# Load model (384-dim, small + fast)
model = SentenceTransformer("all-MiniLM-L6-v2")

def embed_locally(chunks, log_every=25):
    print(f"🔢 Embedding {len(chunks)} chunks locally with 'all-MiniLM-L6-v2'...")

    records = []
    for i, chunk in enumerate(chunks):
        embedding = model.encode(chunk)
        records.append({
            "id": f"chunk-{str(uuid.uuid4())}",
            "content": chunk,
            "embedding": embedding.tolist()
        })

        if (i + 1) % log_every == 0 or (i + 1) == len(chunks):
            print(f"✅ Embedded {i + 1}/{len(chunks)} chunks...")

    print("✅ All embeddings complete.")
    return records

# Run it
embedding_records = embed_locally(text_chunks)

In [ ]:
# # 🧱 Reload .env to ensure the updated GCS bucket name is used

# from dotenv import load_dotenv, find_dotenv

# load_dotenv(find_dotenv(), override=True)  # This loads the .env file

# import os

# # Confirm GCS bucket name
# print(f"GCS Bucket Name: {os.getenv('GCS_BUCKET_NAME')}")

In [ ]:
# 🧱 Cell 5: Save embedding records to JSON and upload to GCS

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)  # This loads the .env file
import os
import json

# Save as JSON (update to .json extension)
json_path = "vector_data.json"
with open(json_path, "w", encoding="utf-8") as f:
    for record in embedding_records:
        json.dump(record, f)
        f.write("\n")

print(f"✅ Saved {len(embedding_records)} records to {json_path}")

# Upload to GCS (new file extension .json)
json_gcs_path = "vector_data/vector_data.json"
upload_file_to_gcs(GCS_BUCKET_NAME, json_path, json_gcs_path)

In [ ]:
# # 🧪 Validate local vector_data.jsonl format and dimensions

# import json

# with open("vector_data.json", "r", encoding="utf-8") as f:
#     for i, line in enumerate(f):
#         try:
#             obj = json.loads(line)
#             assert isinstance(obj["id"], str), "Missing or invalid 'id'"
#             assert isinstance(obj["content"], str), "Missing or invalid 'content'"
#             assert isinstance(obj["embedding"], list), "Missing or invalid 'embedding'"
#             assert len(obj["embedding"]) == 384, f"Invalid dimension: {len(obj['embedding'])}"
#         except Exception as e:
#             print(f"❌ Error on line {i + 1}: {e}")
#             break
#     else:
#         print("✅ All lines are valid and have correct 384-dim embeddings.")

**` Created Vector Index in GCP UI/Console `**

![image](vector-ai-index-creation.JPG)

In [ ]:
# 🧱 Cell: List all available index endpoints in your project/region

from google.cloud import aiplatform

# Make sure init was done earlier
index_endpoints = aiplatform.MatchingEngineIndexEndpoint.list()

print(f"🔍 Found {len(index_endpoints)} endpoint(s):\n")
for ep in index_endpoints:
    print(f"📌 Name       : {ep.resource_name}")
    print(f"    Display   : {ep.display_name}")
    print(f"    Created   : {ep.create_time}")
    print("-" * 50)

In [ ]:
# 🧱 Cell 6: Create MatchingEngineIndexEndpoint with public access
from google.cloud import aiplatform
import time

endpoint_display_name = "acura-mdx-index-endpoint"

# Create the endpoint (do NOT reuse the returned object)
created_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=endpoint_display_name,
    public_endpoint_enabled=True,
    sync=True
)

# Store the name
new_endpoint_resource_name = created_endpoint.resource_name
print(f"✅ Created endpoint: {new_endpoint_resource_name}")

# Sleep to avoid premature NotFound errors
time.sleep(10)

In [ ]:
# 🧱 Cell 7: Reload endpoint and deploy index
# CAUTION LONG RUNNING CELL

import os
import uuid
import time
from dotenv import load_dotenv, find_dotenv
from google.cloud import aiplatform

# Load env vars
load_dotenv(find_dotenv(), override=True)

# Load index resource name from .env
index_resource_name = os.getenv("INDEX_RESOURCE_NAME")

# Reload index and newly created endpoint
index = aiplatform.MatchingEngineIndex(index_resource_name)
index_endpoint = aiplatform.MatchingEngineIndexEndpoint(new_endpoint_resource_name)

# Generate a valid deployed index ID
deployed_index_id = f"acura_mdx_{str(uuid.uuid4())[:8]}"

# Deploy the index
index_endpoint.deploy_index(
    index=index,
    deployed_index_id=deployed_index_id,
    display_name="acura-mdx-deployment"
)

print(f"🚀 Index deployed with ID: {deployed_index_id}")

# Wait briefly to ensure deployment is fully initialized
time.sleep(10)

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os

# Reload updated .env
load_dotenv(find_dotenv(), override=True)

# Load correct endpoint name from .env
index_endpoint_name = os.getenv("INDEX_ENDPOINT_NAME")
print("✅ Using updated endpoint:", index_endpoint_name)
# Load from env
deployed_index_id = os.getenv("DEPLOYED_INDEX_ID")
print(f"📦 Using deployed index ID: {deployed_index_id}")

location=os.getenv("VERTEX_REGION")
print(f"📦 Using location: {location}")

In [ ]:
# 🧱 Cell 8: Embed and query using text-embedding-005 (768-dim vectors)

from vertexai.preview.language_models import TextEmbeddingModel
from google.cloud import aiplatform
import os

# Load the deployed index and endpoint
index = aiplatform.MatchingEngineIndex(os.getenv("INDEX_RESOURCE_NAME"))
index_endpoint = aiplatform.MatchingEngineIndexEndpoint(os.getenv("INDEX_ENDPOINT_NAME"))

# Embed using the correct model for current project + 768-dim
embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-005")
test_query = "How do I enable snow mode in the Acura MDX?"
query_embedding = embedding_model.get_embeddings([test_query])[0].values

# Query
neighbors = index_endpoint.find_neighbors(
    deployed_index_id=os.getenv("DEPLOYED_INDEX_ID"),
    queries=[query_embedding],
    num_neighbors=5,
    return_full_datapoint=True,
)

# Print top results
for i, match in enumerate(neighbors[0], 1):
    print(f"{i}. Score: {match.distance:.4f} | ID: {match.datapoint.datapoint_id}")